In [ ]:
import pandas as pd
import numpy as np

### Title Basics

In [ ]:
basics = pd.read_csv('title.basics.tsv', sep='\t', dtype={'startYear': str})
basics = basics.drop(columns=['originalTitle', 'endYear', 'genres'])

#### Filter out entries that do not have a start year

In [ ]:
has_start_year = basics['startYear'] != '\\N'
basics = basics[has_start_year]

#### Filter out adult titles

In [ ]:
not_adult = basics['isAdult'] == 0
basics = basics[not_adult]
basics = basics.drop(columns=['isAdult'])

#### Filter out entries that aren't movies

In [ ]:
is_movie = basics['titleType'] == 'movie'
basics = basics[is_movie]

In [ ]:
basics

### Principals

In [ ]:
principals = pd.read_csv('title.principals.tsv', sep='\t', dtype={'types': str})
principals = principals.drop(columns=['job', 'ordering'])

#### Only actors or actresses, not director, composer, etc.

In [ ]:
is_actor = (principals['category'] == 'actor') | (principals['category'] == 'actress')
principals = principals[is_actor]
principals = principals.drop(columns=['category'])

In [ ]:
principals

### Name Basics

In [ ]:
names = pd.read_csv('name.basics.tsv', sep='\t')
names = names.drop(columns=['primaryProfession', 'knownForTitles'])

In [ ]:
names

### Results

In [ ]:
merged = basics.merge(principals, on=['tconst'])
merged = merged.merge(names, on=['nconst'])

In [ ]:
results = pd.DataFrame({'Movie Title': merged['primaryTitle'], 'Actor': merged['primaryName'], 'Characters': merged['characters']})

In [ ]:
results